In [ ]:
import numpy as np
import librosa as lba
import soundfile as sf
from random import random

In [ ]:
stimuli_folder = '../stimuli/'

sr = 44100
tone_duration = 200

rise_duration = 5
rise_length = int(rise_duration * sr / 1000)

fade_duration = 25
fade_length = int(fade_duration * sr / 1000)

tones = {
    'A7': 3520,
    'Ds7': 2489.0160,
    'A6': 1760,
    'Ds6': 1244.5080,
    'A5': 880,
    'Ds5': 622.2540,
    'A4': 440,
    'Ds4': 311.1270,
    'A3': 220,
    'Ds3': 155.5635,
    'A2': 110,
    
    'Fs6': 1479.9780,  # Midpoint of high range
    'C4': 261.6256  # Midpoint of low range
}

# Generate Tones

In [ ]:
dur = tone_duration / 1000
for pitch in tones:
    f = tones[pitch]
    
    # Generate fundamental frequency (co)sine wave
    phase = random() * 2 * np.pi
    tone = lba.tone(f, sr=sr, duration=dur, phi=phase)
    
    # Add first three overtones with slope of -6 db (half amplitude) per octave
    for i in range(3):
        phase = random() * 2 * np.pi
        tone += lba.tone(f * (i + 2), sr=sr, duration=dur, phi=phase) / (i + 2)
    
    # Rescale waveform to range [-1, 1] to prevent clipping
    tone /= np.abs(tone).max()

    # Apply linear fade to ending
    tone[-fade_length:] *= np.linspace(1, 0, fade_length)
    # Apply sharp linear rise to start of tone
    tone[:rise_length] *= np.linspace(0, 1, rise_length)
    
    # Save tone
    sf.write('../stimuli/tones/tone%s.wav' % pitch, tone, 44100)

## (Balance tones in Audacity now, then return)

# Generate Sequences

### Main Sequences

In [ ]:
octave_ranges = range(2)
pitch_levels = range(6)
loudness_levels = ('soft', 'normed', 'loud')
iois = [1000, 918, 843,
        774, 710, 652,
        599, 550, 504,
        463, 425, 390,
        358, 329, 302]

for r in octave_ranges:
    
    # Each octave range uses a different set of six pitches
    pitches = ('A2', 'Ds3', 'A3', 'Ds4', 'A4', 'Ds5') if r == 0 else ('Ds5', 'A5', 'Ds6', 'A6', 'Ds7', 'A7')

    for i in pitch_levels:
        
        for j, loudness in enumerate(loudness_levels):
            
            # Load the i-th pitch level in the current octave range at the appropriate loudness
            tone, _ = lba.load(stimuli_folder + 'tones/tone%s-%s.wav' % (pitches[i], loudness), sr=sr)
            
            for ioi in iois:

                # Create array of appropriate length to hold the audio sequence
                ms_ioi = ioi / 1000.
                sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)

                # Insert tones at appropriate locations
                for k in range(5):
                    tone_start = int(np.ceil(k * ms_ioi * sr))
                    tone_end = tone_start + tone.shape[0]
                    sequence[tone_start:tone_end] = tone

                # Cut silence from the end of the sequence
                sequence = np.trim_zeros(sequence, 'b')

                # Save sequence to WAV file (sequence_<octave range>_<pitch level>_<IOI>_<loudness>.wav)
                outfile = stimuli_folder + 'sequence_%i_%i_%i_%i.wav' % (r, i, ioi, j)
                sf.write(outfile, sequence, sr)

### Practice Trials

In [ ]:
for r in octave_ranges:
    
    # Load practice tone for the current octave range
    practice_tone = 'C4' if r == 0 else 'Fs6'
    tone, _ = lba.load(stimuli_folder + 'tones/tone%s-normed.wav' % practice_tone, sr=sr)
    
    for ioi in (407, 550, 741):
        
        # Create array of appropriate length to hold audio sequence
        ms_ioi = ioi / 1000.
        sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)

        # Insert tones at appropriate locations
        for i in range(5):
            tone_start = int(np.ceil(i * ms_ioi * sr))
            tone_end = tone_start + tone.shape[0]
            sequence[tone_start:tone_end] = tone

        # Cut silence from the end of the sequence
        sequence = np.trim_zeros(sequence, 'b')

        # Save sequence to WAV file (practice_sequence_<octave range>_<IOI>.wav)
        outfile = stimuli_folder + 'practice_sequence_%i_%i.wav' % (r, ioi)
        sf.write(outfile, sequence, sr)

### Metronome

In [ ]:
# Load ticking sound
tone, sr = lba.load(stimuli_folder + 'tones/tick.wav', sr=None)

# Create array of appropriate length to hold audio sequence
ioi = 550
ms_ioi = ioi / 1000.
sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)
        
# Insert tones at appropriate locations
for i in range(5):
    tone_start = int(np.ceil(i * ms_ioi * sr))
    tone_end = tone_start + tone.shape[0]
    sequence[tone_start:tone_end] = tone
        
# Cut silence from the end of the sequence
sequence = np.trim_zeros(sequence, 'b')
        
# Save sequence to WAV file
outfile = stimuli_folder + 'reference_sequence.wav'
sf.write(outfile, sequence, sr)